# Project 3, Part 1, Create and load the stations, lines, and travel times tables for the BART system

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering

Students in the group:
* Aris Chalini
* Jack Galvin
* Matt Lauritzen

Year: 2022

Semester: Spring

Section: 09


# Included Modules and Packages

Code cell containing your includes for modules and packages

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Code cells containing any supporting code, such as connecting to the database, any functions, etc.  

Remember you can freely use any code from the labs. You do not need to cite code from the labs.

Some starter code is provided

You may change the starter code as needed

You may add as much code and/or as many code cells as you need

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

# 3.1.1 Drop the stations table if it exists

The stations table should be named stations

In [6]:
connection.rollback()

query = """

drop table if exists stations;

"""

cursor.execute(query)

connection.commit()


# 3.1.2 Drop the lines table if it exists

The lines table should be named lines

In [7]:
connection.rollback()

query = """

drop table if exists lines;

"""

cursor.execute(query)

connection.commit()


# 3.1.3 Drop the travel times table if it exists

The travel times table should be named travel_times

In [8]:
connection.rollback()

query = """

drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()


# 3.1.4 Create the stations table

The stations table should be named stations with the following columns:
* station varchar(32)
* latitude numeric(9,6)
* longitude numeric(9,6)
* transfer_time numeric(3) 

station should be the primary key

This is a normal table, not a staging table

transfer_time is in seconds

In [9]:
connection.rollback()

query = """

create table stations (
  station varchar(32),
  latitude numeric(9,6),
  longitude numeric(9,6),
  transfer_time numeric(3),
  primary key (station)
);

"""

cursor.execute(query)

connection.commit()

# 3.1.5 Create the lines table

The lines table should be named lines with the following columns:

* line varchar(6)
* sequence numeric(2)
* station varchar(32)

line, sequence should be the composite primary key

This is a normal table, not a staging table

The stations in each line are listed in order going one direction. The lines move in both directions.  We will only store the data one way, as this is one of the rules of Third Normal Form (3NF).

In [10]:
connection.rollback()

query = """

create table lines (
  line varchar(6),
  sequence numeric(2),
  station varchar(32),
  primary key (line, sequence)
);

"""

cursor.execute(query)

connection.commit()

# 3.1.6 Create the travel times table

The travel times table should be named travel_times with the following columns:

* station_1 varchar(32)
* station_2 varchar(32)
* travel_time numeric(3) 

station_1, station_2 should be the composite primary key

This is a normal table, not a staging table

Each entry shows the travel time between two stations, with station_1 alphabetically less than station_2.  

Travel time is the same from station_1 to station_2 as it is from station_2 to station_1.

Each line has the same travel time between the same two stations.  

travel_time is in seconds.

In [11]:
connection.rollback()

query = """

create table travel_times (
  station_1 varchar(32),
  station_2 varchar(32),
  travel_time numeric(3),
  primary key (station_1, station_2)
);

"""

cursor.execute(query)

connection.commit()

# 3.1.7 Display the file stations.csv 

Display the first 10 rows in the CSV file using the function my_read_csv_file() from the labs, which has been provided above


In [13]:
my_read_csv_file('../data/stations.csv', 10)

['station', 'latitude', 'longitude', 'transfer_time']
['12th Street', '37.803608', '-122.272006', '282']
['16th Street Mission', '37.764847', '-122.420042', '287']
['19th Street', '37.807869', '-122.26898', '67']
['24th Street Mission', '37.752', '-122.4187', '277']
['Antioch', '37.996281', '-121.783404', '0']
['Ashby', '37.853068', '-122.269957', '299']
['Balboa Park', '37.721667', '-122.4475', '48']
['Bay Fair', '37.697', '-122.1265', '63']
['Berryessa', '37.368361', '-121.874655', '288']

Printed  10 lines of  51 total lines.


# 3.1.8 Display the file lines.csv 

Display the first 10 rows in the CSV file using the function my_read_csv_file() from the labs, which has been provided above

In [14]:
my_read_csv_file('../data/lines.csv', 10)

['line', 'sequence', 'station']
['blue', '1', 'Dublin']
['blue', '2', 'West Dublin']
['blue', '3', 'Castro Valley']
['blue', '4', 'Bay Fair']
['blue', '5', 'San Leandro']
['blue', '6', 'Coliseum']
['blue', '7', 'Fruitvale']
['blue', '8', 'Lake Merritt']
['blue', '9', 'West Oakland']

Printed  10 lines of  115 total lines.


# 3.1.9 Display the file travel_times.csv 

Display the first 10 rows in the CSV file using the function my_read_csv_file() from the labs, which has been provided above

In [15]:
my_read_csv_file('../data/travel_times.csv', 10)

['station_1', 'station_2', 'travel_time']
['12th Street', '19th Street', '120']
['12th Street', 'Lake Merritt', '180']
['12th Street', 'West Oakland', '300']
['16th Street Mission', '24th Street Mission', '120']
['16th Street Mission', 'Civic Center', '180']
['19th Street', 'MacArthur', '180']
['24th Street Mission', 'Glen Park', '180']
['Antioch', 'Pittsburg Center', '420']
['Ashby', 'Downtown Berkeley', '180']

Printed  10 lines of  52 total lines.


# 3.1.10 Load stations data into database table

Load the CSV file, stations.csv, into the database table stations

In [16]:
connection.rollback()

query = """

copy stations
from '/user/projects/ucb_mids_project_3/data/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

# 3.1.11 Load lines data into database table

Load the CSV file, lines.csv, into the database table lines


In [17]:
connection.rollback()

query = """

copy lines
from '/user/projects/ucb_mids_project_3/data/lines.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

# 3.1.12 Load travel_times data into database table

Load the CSV file, travel_times.csv, into the database table travel_times

In [18]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/ucb_mids_project_3/data/travel_times.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

# 3.1.13 Verify the stations loaded correctly

Write a query to verify the stations loaded correctly

Select all columns and all rows from the table stations

Sort by station

Display the results in a Pandas data frame

In [19]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from stations
order by station

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0
5,Ashby,37.853068,-122.269957,299
6,Balboa Park,37.721667,-122.447500,48
7,Bay Fair,37.697000,-122.126500,63
8,Berryessa,37.368361,-121.874655,288
9,Castro Valley,37.690748,-122.075679,0


# 3.1.14 Verify the lines loaded correctly

Write a query to verify the lines loaded correctly

Select all columns and all rows from the table lines

Sort by lines, sequence

Display the results in a Pandas data frame

In [20]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from lines
order by lines, sequence

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,sequence,station
0,blue,1,Dublin
1,blue,2,West Dublin
2,blue,3,Castro Valley
3,blue,4,Bay Fair
4,blue,5,San Leandro
...,...,...,...
109,yellow,23,Daly City
110,yellow,24,Colma
111,yellow,25,South San Francisco
112,yellow,26,San Bruno


# 3.1.15 Verify the travel times loaded correctly

Write a query to verify the travel times loaded correctly 

Select all columns and all rows from the travel_times

Sort by station_1, station_2

Display the results in a Pandas data frame

In [21]:
rollback_before_flag = True
rollback_after_flag = True

query = """

select *
from travel_times
order by station_1, station_2

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station_1,station_2,travel_time
0,12th Street,19th Street,120
1,12th Street,Lake Merritt,180
2,12th Street,West Oakland,300
3,16th Street Mission,24th Street Mission,120
4,16th Street Mission,Civic Center,180
5,19th Street,MacArthur,180
6,24th Street Mission,Glen Park,180
7,Antioch,Pittsburg Center,420
8,Ashby,Downtown Berkeley,180
9,Ashby,MacArthur,240
